<a href="https://colab.research.google.com/github/Arddistic/Arddistic/blob/main/disEZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import threading
import streamlit as st
import pickle
import time
import pandas as pd
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
from dotenv import load_dotenv
load_dotenv('kk.env')

In [ ]:
st.title("Brief summary of disease pathology")
st.sidebar.title("URLs à analyser")

urls = []
for i in range(3):
    url = st.sidebar.text_input(f"URL {i+1}")
    urls.append(url)


process_url_clicked = st.sidebar.button("Analyze Data")
#file_path = "faiss_store_openai.pkl"

main_placeholder = st.empty()
llm = OpenAI(temperature=0.9, max_tokens=500)


In [ ]:
if process_url_clicked:

    loader = UnstructuredURLLoader(urls=urls)
    main_placeholder.text("✅")
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n', '.', ','],
        chunk_size=1000
    )
    main_placeholder.text("✅✅")
    docs = text_splitter.split_documents(data)
    # embeddings
    embeddings = OpenAIEmbeddings()
    vectorstore_openai = FAISS.from_documents(docs, embeddings)
    main_placeholder.text("✅✅✅")
    time.sleep(2)

    vectorstore_openai.save_local("faiss_store");


In [ ]:
#query = main_placeholder.text_input("Question: ")

queries = [' State the disease name',
           'How does the disease spread?',
           'What type of disease is it?',
         'List most common symptoms',
          'Who are most prone to this disease?',
         'State when to see a doctor',
         'List three treatment options']

results = [];

for query in queries:
    #if os.path.exists(file_path):
        #with open(file_path, "rb") as f:
            #vectorstore = pickle.load(f)
            vectorstore = FAISS.load_local("faiss_store", OpenAIEmbeddings(), allow_dangerous_deserialization=True)
            chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())
            result = chain({"question": query}, return_only_outputs=True)
            # result will be a dictionary of this format --> {"answer": "", "sources": [] }
            #st.header("Answer")
            #st.write(result["answer"])
            answer = result["answer"]
            # sources = result.get("sources", "")
            #results.append({"Question": query, "Answer": answer, "Sources": sources})
            max_answer_length = 75  # Adjust this value according to your preference
            truncated_answer = (answer[:max_answer_length] + '...') if len(answer) > max_answer_length else answer
            #sources = result.get("sources", "")
            results.append({"Question": query, "Answer": truncated_answer})

            # Display sources, if available
            #sources = result.get("sources", "")
            #if sources:
                #st.subheader("Sources:")
               # sources_list = sources.split("\n")  # Split the sources by newline
                #for source in sources_list:
                   # st.write(source)



df = pd.DataFrame(results) #convert results to dataframe

custom_width = 750
custom_height = 450
st.dataframe(df, width=custom_width, height=custom_height) #display dataframe
